In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
subh = np.loadtxt('../_data/subhalos.dat')

halo_mask, = np.where((subh[:,15]==0) & (subh[:,16]==0))

subh_name = subh[:,0][halo_mask]
subh_ind  = np.int_(subh_name)
# nsubh = len(subh_name)

print(subh_ind)

[ 0  1  3  4  5  6  7  9 20 21 22 27 28 31 33 35 36 37 38 39 41 42 45 46
 49 50 51 53 54 64 65 66 69 73 74 79 84 85 87 88 90 91 92 95 96]


In [ ]:
import matplotlib.pyplot as plt
import h5py
import rotation_mio as rot
import numpy as np
import time_conversion as time
import bines2 as bines

path = '/z/omarioni/gradients/_simulations/snap_127/'

for j in subh_ind[:8]:
    
    snap = h5py.File(path + 'subhalo_'+str('%03d'%j)+'.h5py', 'r')

    aexp = snap['/Time'][()]
    h    = snap['/h'][()]
    Om_L = snap['/Omega_Lambda'][()]
    Om_M = snap['/Omega_0'][()]

    pstr = snap['/Str/Coordinates'][()]
    mstr = snap['/Str/Masses'][()]
    vstr = snap['/Str/Velocities'][()]

    met  = snap['/Str/Metallicity'][()]
    sft  = snap['/Str/FormationTime'][()]

    age_ = time.conv2(sft, h, Om_L, Om_M)
    aa   = ~np.isnan(age_)
    age_ = age_[aa]
    age  = np.nanmax(age_) - age_
    
    FeH_ = met[:,8]/met[:,0]
    FeH  = FeH_/0.0127

    xstr = pstr[:,0]*aexp/h
    ystr = pstr[:,1]*aexp/h
    zstr = pstr[:,2]*aexp/h
    rstr = np.sqrt(xstr**2+ystr**2+zstr**2)

    v_x = vstr[:,0] *np.sqrt(aexp)
    v_y = vstr[:,1] *np.sqrt(aexp)
    v_z = vstr[:,2] *np.sqrt(aexp)

    #----------------------masas----------------------------
    mstr = mstr/h

    veloc,=np.where(rstr < 3)

    #----------componentes de la velocidad del centro de masa------------
    vxcm = sum(mstr[veloc]*v_x[veloc])/sum(mstr[veloc])
    vycm = sum(mstr[veloc]*v_y[veloc])/sum(mstr[veloc])
    vzcm = sum(mstr[veloc]*v_z[veloc])/sum(mstr[veloc])

    vx = v_x - vxcm
    vy = v_y - vycm
    vz = v_z - vzcm

    e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vx,vy,vz,5)

    xn_str = e1x*xstr + e1y*ystr + e1z*zstr
    yn_str = e2x*xstr + e2y*ystr + e2z*zstr
    zn_str = e3x*xstr + e3y*ystr + e3z*zstr

    Rstr = np.sqrt(xn_str**2 + yn_str**2)
    
    Rstr   = Rstr[aa]
    zn_str = zn_str[aa]
    mstr   = mstr[aa]
    FeH    = FeH[aa]

    age_4,  = np.where((age[mask] > 3.5) & (age[mask] < 4.5))
    age_8,  = np.where((age[mask] > 7.5) & (age[mask] < 8.5))
    age_10, = np.where((age[mask] > 9.5) & (age[mask] < 10.5))
    
    fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(16, 8))
    fig.subplots_adjust(bottom=0.10, left =0.12, right = 0.97, top = 0.97)

    ax[0,0].set_ylim(-5,5)
    ax[0,0].set_xlim(0,25)
    ax[0,0].minorticks_on()
    ax[0,0].tick_params( labelsize=15)
    ax[0,0].tick_params('both', length=5, width=1.2,which='minor', direction='in', right='on',top='on')
    ax[0,0].tick_params('both', length=8, width=1.2,which='major', direction='in', right='on',top='on')
    ax[0,0].set_xlabel('R [kpc]',fontsize=15)
    ax[0,0].set_ylabel('z [kpc]',fontsize=15)
#     fig.savefig('../_imagenes/star_ages_'+str('%03d'%j)+'.png',dpi=75, xxbox_inches='tight')